# CSFQ class

In [9]:
%matplotlib
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
from math import pi
from scipy.constants import h,e, hbar

class CSFQ(object):
    '''
    compute eigenvalues and eigenstates of CSFQ Hamilitonian using 1D potential
    Args:
           I0 : critical current of large JJ in A
           alpha : critical current ratio of small to large JJ, < 1
           Cj : junction capacitance of large JJ in F
           Cs : shunt capacitance in F
    '''
    
    def __init__(self, I0=None, alpha=None, Cj=None, Cs=None):
        
        self.I0 = I0
        self.alpha = alpha 
        self.Cj = Cj
        self.Cs = Cs
            
    def solveHamiltonian(self, flux):
        """
        Solve Hamiltonian.
        Args:
            flux: normalized by flux quantum
        Return:
            Tuple of sorted eigenvalues and eigenvectors
        """
        f0 = h/2/e; #flux quantum
        n = 256 # dimension of Hamitonian matrix
    
        beta = self.alpha + self.Cs / self.Cj
        
        # Do 1D Potential
        delta = np.linspace(-1.8*pi, 1.8*pi, n)
        dd = delta[1] - delta[0]

        U = self.potentialE(delta, flux)

        # Numerically sovle 1D potential
        Cnew = (2*self.Cj/4) * (1+2*beta)
        h22m = 4*e**2/2/Cnew
        diffOp = -h22m/dd**2
        onOp = +2*h22m/dd**2
        H = np.diag(U)+ np.eye(len(U), k=1) * diffOp + np.eye(len(U), k=-1) * diffOp + np.eye(len(U)) * onOp
        self.H = H
        
        eigval, eigvec = LA.eigh(H)
        
        eigval_sorted = np.sort(eigval)  # eigval.sort()
        eigvec_sorted = eigvec[:, eigval.argsort()]
        
        energy_diff = np.diff(eigval_sorted) / h / 1e9 # in GHz

        return (energy_diff, eigvec_sorted)
    
    def potentialE(self, delta, f):
        """
        f: flux normalized by flux quantum 
        """
        f0 = h/2/e
        n = 256 # dimension of Hamitonian matrix
    
        beta = self.alpha + self.Cs / self.Cj
        
        U = -2*self.I0*f0/2/pi*np.cos(delta/2) - self.alpha*self.I0*f0/2/pi*np.cos(2*pi*f-2*delta/2)
          
        return U

Using matplotlib backend: TkAgg


In [32]:
# Test
I0 =200e-9
Cj = 5e-15
Cs = 60e-15
alpha = 0.43
flux = 0.5

csfq = CSFQ(I0=I0, Cj=Cj, Cs=Cs, alpha=alpha)
energy_diff, eigenvec = csfq.solveHamiltonian(flux)

print(f'f01 = {energy_diff[0]:.3f} GHz')
print(f'anharm = {energy_diff[1] - energy_diff[0]:.3f} GHz')

f01 = 4.935 GHz
anharm = 0.608 GHz


# Spectrum vs flux

In [30]:
# get spectrum up to w34
I0 =200e-9
Cj = 5e-15
Cs = 55e-15
alpha = 0.43

csfq = CSFQ(I0=I0, Cj=Cj, Cs=Cs, alpha=alpha)

N = 201 # number of flux points
flux = np.linspace(0,1,N)

# plot set up
f, spec = np.array([]), np.array([])
fig, axes = plt.subplots(1,1)
lines1, = axes.plot([], [],'k-')
lines2, = axes.plot([], [], 'r-')
lines3, = axes.plot([], [], 'b-')
lines4, = axes.plot([], [], 'g-')
axes.set_xlabel(r'Flux ($\Phi_0$)', size=18)
axes.set_ylabel('Energy (GHz)', size=18)
axes.tick_params(axis='x', labelsize=14)
axes.tick_params(axis='y', labelsize=14)
axes.grid(which='both',color='gray', linestyle='--', linewidth=0.5)

for f1 in flux:
    energy_diff = csfq.solveHamiltonian(f1)[0][0:3]
    if spec.size ==0:
        spec = energy_diff
        spec.shape = (1,spec.size)
    else:
        spec = np.vstack((spec, energy_diff))
        
    f = np.append(f,f1)
        
    lines1.set_xdata(f)
    lines1.set_ydata(spec[:,0])
    lines2.set_xdata(f)
    lines2.set_ydata(spec[:,1])
    lines3.set_xdata(f)
    
    lines3.set_ydata(spec[:,2])
      
    axes.relim()
    axes.autoscale()
    
    fig.canvas.draw() 
    fig.canvas.flush_events()

axes.legend((f'f01', f'f12', f'f23'), loc=0, fontsize=16, frameon=False)
f01 = csfq.solveHamiltonian(0.5)[0][0]                                
f12 = csfq.solveHamiltonian(0.5)[0][1]  

print('f01={:.4g},anharmonicity={:.3g}'.format(f01, f12-f01))

f01=5.164,anharmonicity=0.648
